In [1]:
# play audio in ipython notebook
try:
    import winsound
except ImportError:
    !pip install winsound
    import winsound
duration = 1000  # milliseconds
freq = 440  # Hz

try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime

In [2]:
import numpy as np
import time
import requests
import os
import pickle
from lxml import html
import numpy as np
import urllib.request
import face_recognition
import cv2
import matplotlib.pyplot as plt
import pytesseract
from googlesearch import search
from PIL import Image, ImageStat
import time
tt = time.time
st = tt()

time: 7.87 s


In [3]:
# get prof name from thumbnail
def save_as_pickle(file_path, file):
    with open(file_path, 'wb') as f:
        pickle.dump(file, f)

def load_from_pickle(file_path):
    with open(file_path, 'rb') as f:
        file = pickle.load(f)
    return file

def plot_image_list(img_list, figsize=(12, 12), subplot_n_cols=2):
    plt.figure(figsize=(12, 12))
    for i,img in enumerate(img_list):
        plt.subplot(int(len(img_list)/subplot_n_cols)+1, subplot_n_cols, i+1)
        try:
            plt.imshow(img)
        except:
            pass
        
def brightness(img):
    img = Image.fromarray(img)
    img = img.convert('L')
    stat = ImageStat.Stat(img)
    return stat.rms[0]

def is_valid_img(img, lthresh = 100, hthresh = 200):
    bright = brightness(img)
    print(bright)
    if bright<lthresh or bright>hthresh:
        return False
    else:
        return True
        
def get_extension_files(path_to_json, ext='.json'):
    json_files = [os.path.join(path_to_json, pos_json) for pos_json in os.listdir(path_to_json) if pos_json.endswith(ext)]
    return json_files

def flatten_list(l):
    return [item for sublist in l for item in sublist]

def post_processing(img):
    norm_img = np.zeros((img.shape[0], img.shape[1]))
    norm_img = cv2.normalize(img, norm_img, 0, 255,
                             cv2.NORM_MINMAX)
    return img

time: 10.5 ms


In [ ]:
base_dir = 'C:\\Users\\YASH\\nptel_face_recognition'
data_dir = os.path.join(base_dir, 'data')
results_dir = os.path.join(base_dir, "results")
face_recognition_results_dir = os.path.join(results_dir, "face_recognition")
os.makedirs(face_recognition_results_dir, exist_ok = True)

known_faces_dir = os.path.join(data_dir, 'known_faces')
full_vedios_dir = os.path.join(data_dir, 'full_vedio')
pkl_files_dir = os.path.join(data_dir, "pkl_files")

folder_name_2_prof_name_dict = load_from_pickle(os.path.join(pkl_files_dir, "folder_name_2_prof_name_dict.pkl"))

images_dir_list = [x[0] for x in os.walk(known_faces_dir)][1:]
known_face_names = [folder_name_2_prof_name_dict[int(w.split('\\')[-1])] for w in images_dir_list]

known_face_images_nested_list = []
for image_dir in images_dir_list:
    known_images_path_list = get_extension_files(image_dir, ext='.jpg') 
    known_face_images_nested_list.append([face_recognition.load_image_file(w) for w in known_images_path_list])

filter_idx_list=[]
known_face_encodings=[]
known_face_seprate_encodings=[]
for i,(n,nl) in enumerate(zip(known_face_names, known_face_images_nested_list)):
    enc_list = []
    for img in nl:
        enc = face_recognition.face_encodings(img)
        if enc!=[]:
            enc_list.append(enc[0])
    
    seprate_img_arr = np.array(enc_list)
    img_arr = np.array(enc_list).mean()
    if enc_list!=[]:
        known_face_encodings.append(img_arr)
        known_face_seprate_encodings.append(seprate_img_arr)
    else:
        print(f"encoding not found for {n}")
        filter_idx_list.append(i)
known_face_names = [known_face_names[i] for i in range(len(known_face_names)) if i not in filter_idx_list]

winsound.Beep(freq, duration)

In [ ]:
# for 1 vedio

vedio_path = 'C:\\Users\\YASH\\nptel_face_recognition\\full_vedio\\Introduction to Machine Learning\\dYMCwxgl3vk.mp4'
vfname = vedio_path.split('\\')[-1]
print(f"vedio: {vfname}")
video_capture = cv2.VideoCapture(vedio_path)

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

vst = 20
vet = 25
fps = int(video_capture.get(cv2.CAP_PROP_FPS))
st = tt()
i=-1
img_list = []
while video_capture.isOpened():
    i+=1
    if int(i/fps) < vst:
        ret, frame = video_capture.read()
        continue

    if int(i/fps) > vet:
        break

     # Grab a single frame of video
    ret, frame = video_capture.read()

    # Resize frame of video to 1/4 size for faster face recognition processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = small_frame[:, :, ::-1]

    # process
    NUM_FRAMES_PER_SEC = 1
    if i%int(fps/NUM_FRAMES_PER_SEC)==0:
        face_locations = face_recognition.face_locations(rgb_small_frame)
        if not len(face_locations)==1:
            continue

        face_encoding = face_recognition.face_encodings(rgb_small_frame, face_locations)[0]

        # face distances with average encodings
        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)

        # face distances as sum of distances for all faces of a prof
#         face_distances=[]
#         for known_face_encodings_list in known_face_seprate_encodings:
#             face_distances.append(face_recognition.face_distance(known_face_encodings_list, face_encoding).mean())

        best_match_index = np.argmin(face_distances)
        if face_distances[best_match_index]<threshold:
            name = 'unknown'
        else:
            name = known_face_names[best_match_index]

        names_list.append(name)
        tp_dir = os.path.join(face_recognition_results_dir, "tp")
        fp_dir = os.path.join(face_recognition_results_dir, "fp")
        os.makedirs(tp_dir, exist_ok = True)
        os.makedirs(fp_dir, exist_ok = True)

        if name==true_name:
            y_pred.append(1)
            image_path = os.path.join(tp_dir, name+'.jpg')
            cv2.imwrite(image_path, img)
        else:
            y_pred.append(0)
            image_path = os.path.join(fp_dir, name+'.jpg')
            cv2.imwrite(image_path, img)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()